# Note:
* First of all, **run the libraries**
* Set the path for ** *CSV-file*** 
* Then **load the file (*CSV*)** by the following instructions provided after running 
* Continue your work ...

# Libraries

In [ ]:
import filterpy.kalman as kf
import filterpy.stats as stats

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

from urllib.request import urlopen
import xml.etree.ElementTree as et 
import math
from datetime import datetime
from decimal import Decimal

# Functions

In [ ]:
def difference_in_unit(item1, item2, unit="distance"):
    return (item2-item1)

In [ ]:
# function to convert to subscript
def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s))
    return x.translate(res)

In [ ]:
# function to convert to superscript
def get_super(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    super_s = "ᴬᴮᶜᴰᴱᶠᴳᴴᴵᴶᴷᴸᴹᴺᴼᴾQᴿˢᵀᵁⱽᵂˣʸᶻᵃᵇᶜᵈᵉᶠᵍʰᶦʲᵏˡᵐⁿᵒᵖ۹ʳˢᵗᵘᵛʷˣʸᶻ⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾"
    res = x.maketrans(''.join(normal), ''.join(super_s))
    return x.translate(res)

In [ ]:
def detect_direction(angle=0.00): 
    if (angle == 0.00): 
        return "None", "North" 
    elif (angle>0.00 and angle<90.00): #
        return "East", "North" 
    elif (angle == 90.00): 
        return "East", "None" 
    elif (angle>90.00 and angle<180.00): #
        return "East", "South" 
    elif (angle == 180.00): 
        return "None", "South" 
    elif (angle>180.00 and angle<270.00): #
        return "West", "South" 
    elif (angle == 270.00): 
        return "West", "None" 
    elif (angle>270.00 and angle<360.00): #
        return "West", "North" 

In [ ]:
def find_min_max_weight(x_dir, y_dir): 
    if (x_dir == "East" and y_dir == "North"): 
        return 0, 90
    elif (x_dir == "East" and y_dir == "South"): 
        return 90, 180
    elif (x_dir == "West" and y_dir == "South"): 
        return 180, 270
    elif (x_dir == "West" and y_dir == "North"): 
        return 270, 360
    else: 
        return 0.0, 0.0

In [ ]:
def preaict(x, y, a, s): 
    x_dir , y_dir = detect_direction(a)
    minweight_angle, maxweight_angle = find_min_max_weight(x_dir, y_dir)
    weight_range = maxweight_angle-minweight_angle
    xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
    print ("{:}, {:}, {:.2f}, {:.2f}, {:.2f}".format(
        x_dir, y_dir, a, xweight_angle, yweight_angle
    ))
    if (x_dir == "None" or y_dir == "None"): 
        if (x_dir == "None"): 
            yweight_angle = 1
            if (y_dir == "North"): 
                #print ("None-North: {:.2f}".format(a))
                x_pred, y_pred = x, y+yweight_angle*dt*s
            elif (y_dir == "South"): 
                #print ("None-South: {:.2f}".format(a))
                x_pred, y_pred = x, y-yweight_angle*dt*s
        elif (y_dir == "None"): 
            xweight_angle = 1
            if (x_dir == "East"): 
                #print ("East-None: {:.2f}".format(a))
                x_pred, y_pred = x+yweight_angle*dt*s, y
            elif (x_dir == "West"): 
                #print ("West-None: {:.2f}".format(a))
                x_pred, y_pred = x-yweight_angle*dt*s, y
    elif (x_dir == "East" and y_dir == "North"): 
        #print ("East-North: {:.2f}".format(a))
        xweight_angle, yweight_angle = (a-minweight_angle)/weight_range, (maxweight_angle-a)/weight_range
        x_pred, y_pred = x+xweight_angle*dt*s, y+yweight_angle*dt*s
    elif (x_dir == "East" and y_dir == "South"): 
        #print ("East-South: {:.2f}".format(a))
        xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
        x_pred, y_pred = x+xweight_angle*dt*s, y-yweight_angle*dt*s
    elif (x_dir == "West" and y_dir == "South"): 
        #print ("West-South: {:.2f}".format(a))
        xweight_angle, yweight_angle = (a-minweight_angle)/weight_range, (maxweight_angle-a)/weight_range
        x_pred, y_pred = x-xweight_angle*dt*s, y-yweight_angle*dt*s
    elif (x_dir == "West" and y_dir == "North"): 
        #print ("West-North: {:.2f}".format(a))
        xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
        x_pred, y_pred = x-xweight_angle*dt*s, y+yweight_angle*dt*s
    return round(x_pred, 2), round(y_pred, 2)

In [ ]:
def distance_xy_between_two_points(x_actual, y_actual, x_pred, y_pred): 
    distance = math.sqrt(
        ((y_actual-y_pred)**2)
        +
        ((x_actual-x_pred)**2)
    )
    return distance

In [ ]:
def distance_xy_between_two_array(
    xy_array_actual, xy_array_pred, actual_cols=["x", "y"], pred_cols=["x", "y"]
): 
    xy_array_actual = xy_array_actual.reset_index()
    distance_rows = []
    distance_cols = ["distance"]
    n = xy_array_actual.shape[0]
    for index, coordinate in xy_array_actual.iterrows(): 
        x_actual = coordinate[actual_cols[0]]
        y_actual = coordinate[actual_cols[1]]
        x_pred = xy_array_pred[pred_cols[0]].values[index]
        y_pred = xy_array_pred[pred_cols[1]].values[index]
        
        distance = distance_xy_between_two_points(
            x_actual, y_actual, x_pred, y_pred
        )
        distance_rows.append({
            "distance": distance
        })
    new_df = pd.DataFrame(distance_rows, columns = distance_cols)
    return new_df

In [ ]:
def error_xy_array(veh1503_records_actual1, veh1503_records_pred1): 
    n = veh1503_records_actual1.shape[0]
    total_difference = 0
    distance_array = distance_xy_between_two_array(
        veh1503_records_actual1, veh1503_records_pred1, ["x", "y"], ["xp", "yp"]
    )
    return distance_array

In [ ]:
def root_mean_square_error(error_data): 
    number_of_entries = error_data.shape[0] 
    sum_of_entries = error_data["distance"].sum() 
    error_mean = math.sqrt(sum_of_entries/number_of_entries) 
    print ("RMSE: ", error_mean) 

# Setting the path from where we'll load the CSV-file

In [ ]:
path_to_csv_file = (
    r"/media/ncl-akraino/Volume-HDD-1/akraino/ws-akraino-datasets/OpenStreetMap 2 Traces/" 
    "SelfOpenStreetMapTraceForASparseTraffic.csv" 
)

# Initializing variables

In [ ]:
temp_counter = 0

In [ ]:
MARKER_SZ = 7

In [ ]:
dt = 1

In [ ]:
display_max_rows = 60
display_max_cols = 10

In [ ]:
pd.set_option("display.max_columns", display_max_cols)

<br>

## Displaying (wink) Copyrights information

In [ ]:
# Copyrights by Asif Mehmood
print ("Asif©"+datetime.now().strftime("%Y"))

<br>

## Creating the pandas dataframe from CSV-file

In [ ]:
df_org = pd.read_csv(path_to_csv_file)

## Converting strings to float-values in the dataset

In [ ]:
df_org["time"] = df_org["time"].astype(float)
df_org["x"] = df_org["x"].astype(float)
df_org["y"] = df_org["y"].astype(float)
df_org["angle"] = df_org["angle"].astype(float)
df_org["speed"] = df_org["speed"].astype(float)

In [ ]:
df_org = df_org.drop(
    columns=["lane", "pos"]
)

## Fetching the maximum, minimum coordinates

In [ ]:
min_x = min(df_org["x"])
min_y = min(df_org["y"])
max_x = max(df_org["x"])
max_y = max(df_org["y"])
print ("min/max longititude (X): ", min_x, ", ", max_x)
print ("min/max latitude (Y):\t ", min_y, ", ", max_y)

## Setting maximum, minimum limit for graphs in Pilot

In [ ]:
plt.xlim([math.floor(min_x), math.ceil(max_x)])
plt.ylim([math.floor(min_y), math.ceil(max_y)])
plt.figure(figsize=(12, 8))

In [ ]:
veh1501_records = df_org[df_org["id"] == "veh1501"]

### Plot longitude/latitude for vehicle 1501

In [ ]:
veh1501_plot = veh1501_records.plot(x="x", y="y", label="Vehicle 1501", figsize=(12,8))
veh1501_plot.legend("Vehicle 1501's mobility on map")
veh1501_plot.set_xlabel("longitude")
veh1501_plot.set_ylabel("latitude")
veh1501_plot.set_xlim([math.floor(min_x), math.ceil(max_x)])
veh1501_plot.set_ylim([math.floor(min_y), math.ceil(max_y)])

In [ ]:
### Plot x/time for vehicle 1501

In [ ]:
veh1501_plot2 = veh1501_records.plot(x="x", y="time", label="Vehicle 1501", figsize=(12,8))
veh1501_plot2.legend("Vehicle 1501's mobility on map")
veh1501_plot2.set_xlabel("x")
veh1501_plot2.set_ylabel("time")
veh1501_plot2.set_xlim([math.floor(min_x), math.ceil(max_x)])
veh1501_plot2.set_ylim([math.floor(min_y), math.ceil(max_y)])

### KF testing (start)

In [ ]:
kf.predict(x=10., P=3., u=1., Q=4.)

In [ ]:
stats.plot_gaussian_pdf(
    mean=10., 
    variance=1., 
    xlim=(4, 16), 
    ylim=(0, .5) 
);

In [ ]:
xs = range(500)
ys = np.random.randn(500)*1. + 10.
plt.plot(xs, ys)
print(f'Mean of readings is {np.mean(ys):.3f}')

In [ ]:
xs = range(500)

In [ ]:
type(xs)

In [ ]:
ys = np.random.randn(500)*1. +10
type(ys)

In [ ]:
type(np.random.randn(500)*1)

In [ ]:
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

In [ ]:
g1 = gaussian(3.4, 10.1)
g2 = gaussian(mean=4.5, var=0.2**2)
print(g1)
print(g2)

### KF testing (end)

## Explaining the demonstration of prediction in different situations

In [ ]:
veh1501_records_temp = veh1501_records[6780:6830]

In [ ]:
veh1501_records_temp

In [ ]:
veh1501_records_temp1 = veh1501_records_temp[veh1501_records_temp["time"] == 8047]
veh1501_records_temp2 = veh1501_records_temp[veh1501_records_temp["time"] == 8048]
veh1501_records_temp3 = veh1501_records_temp[veh1501_records_temp["time"] == 8049]

veh1501_records_temp4 = veh1501_records_temp[veh1501_records_temp["time"] == 8063]
veh1501_records_temp5 = veh1501_records_temp[veh1501_records_temp["time"] == 8064]
veh1501_records_temp6 = veh1501_records_temp[veh1501_records_temp["time"] == 8065]

In [ ]:
fig = plt.figure(figsize=(12,8))

plt.plot(
    veh1501_records_temp["x"], 
    veh1501_records_temp["y"], 
    label="Temp: Vehicle 1501", 
    marker=".", 
    markersize=MARKER_SZ 
)

plt.plot(
    veh1501_records_temp1["x"], 
    veh1501_records_temp1["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="\u0394(\u03F4) > 20\u00B0"
)

plt.plot(
    veh1501_records_temp2["x"], 
    veh1501_records_temp2["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    #label="\u0394(\u03F4) > 20\u00B0"
)

plt.plot(
    veh1501_records_temp3["x"], 
    veh1501_records_temp3["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    #label="\u0394(\u03F4) > 20\u00B0"
)

plt.plot(
    veh1501_records_temp4["x"], 
    veh1501_records_temp4["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    #label="\u0394(\u03F4) > 20\u00B0"
)

plt.plot(
    veh1501_records_temp5["x"], 
    veh1501_records_temp5["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    #label="\u0394(\u03F4) > 20\u00B0"
)

plt.plot(
    veh1501_records_temp6["x"], 
    veh1501_records_temp6["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=.7, 
    linewidth=1.5, 
    #label="\u0394(\u03F4) > 20\u00B0"
)

plt.legend(loc='best', fontsize=14)

### 1(a) Finding new location between two consecutive coordinates via angle

In [ ]:
print (veh1501_records_temp1)
print (veh1501_records_temp2)

In [ ]:
c1_dif_x = difference_in_unit(veh1501_records_temp1["x"].values[0], veh1501_records_temp2["x"].values[0])
c1_dir_x = "East" if c1_dif_x>0 else "West"
c1_dif_y = difference_in_unit(veh1501_records_temp1["y"].values[0], veh1501_records_temp2["y"].values[0])
c1_dir_y = "North" if c1_dif_y>0 else "South"
c1_dif_a = difference_in_unit(veh1501_records_temp1["angle"].values[0], veh1501_records_temp2["angle"].values[0])
c1_dif_s = difference_in_unit(veh1501_records_temp1["speed"].values[0], veh1501_records_temp2["speed"].values[0])

print (
    "Direction:\t{:}-{:}".format(c1_dir_y, c1_dir_x)
)
print (
    "(m)\t", "{:.20f}\t\t".format(c1_dif_x), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp2["x"].values[0], veh1501_records_temp1["x"].values[0]), 
)
print (
    "(m)\t", "{:.20f}\t".format(c1_dif_y), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp2["y"].values[0], veh1501_records_temp1["y"].values[0]), 
)
print (
    "(\u00B0)\t", "{:.20f}\t".format(c1_dif_a), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp2["angle"].values[0], veh1501_records_temp1["angle"].values[0]), 
)
print (
    "(m/s)\t", "{:.20f}\t\t".format(c1_dif_s), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp2["speed"].values[0], veh1501_records_temp1["speed"].values[0]), 
)

In [ ]:
c1_point_1 = [
    veh1501_records_temp1["x"].values[0], 
    veh1501_records_temp1["y"].values[0], 
    veh1501_records_temp1["speed"].values[0], 
    veh1501_records_temp1["angle"].values[0] 
]
c1_point_2 = [
    veh1501_records_temp2["x"].values[0], 
    veh1501_records_temp2["y"].values[0], 
    veh1501_records_temp2["speed"].values[0], 
    veh1501_records_temp2["angle"].values[0] 
]

if(c1_dir_y=="South"):
    c1_point_3_y = c1_point_2[1]
    print ("extended-point (y) should be taken from point_2")
if(c1_dir_y=="North"):
    c1_point_3_y = c1_point_2[1]
    print ("extended-point (y) should be taken from point_1")

if(c1_dir_x=="East"):
    c1_point_3_x = c1_point_1[0]
    print ("extended-point (x) should be taken from point_1")
if(c1_dir_x=="West"):
    c1_point_3_x = c1_point_1[0]
    print ("extended-point (x) should be taken from point_2")

c1_point_3 = [c1_point_1[0], c1_point_2[1]]
print (c1_point_3)
c1_point_3 = [c1_point_3_x, c1_point_3_y]
print (c1_point_3)

In [ ]:
c_fig_2ps, ax_2ps = plt.subplots(
    2, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

c1_fig_2ps_g1_p1, = ax_2ps[0].plot(
    c1_point_1[0], 
    c1_point_1[1], 
    "ro", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+1)
)

c1_fig_2ps_g1_p2, = ax_2ps[0].plot(
    c1_point_2[0], 
    c1_point_2[1], 
    "go", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+2)
)

c1_fig_2ps_g1_p3, = ax_2ps[0].plot(
    c1_point_3[0], 
    c1_point_3[1], 
    "ys", 
    markersize=MARKER_SZ+4, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="self-extended point", 
)

c1_fig_2ps_g1_p4, = ax_2ps[0].plot(
    [c1_point_2[0], c1_point_3[0]], 
    [c1_point_2[1], c1_point_3[1]], 
    "c", 
    label="base: {:.2f}m".format(abs(c1_point_3[0]-c1_point_2[0])) 
)

c1_fig_2ps_g1_p5, = ax_2ps[0].plot(
    [c1_point_1[0], c1_point_3[0]], 
    [c1_point_1[1], c1_point_3[1]], 
    "m", 
    label="perpendicular: {:.2f}m".format(abs(c1_point_3[1]-c1_point_1[1])) 
)

c1_fig_2ps_g1_p6, = ax_2ps[0].plot(
    [c1_point_1[0], c1_point_2[0]], 
    [c1_point_1[1], c1_point_2[1]], 
    "r", 
    label="hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c1_point_3[1]-c1_point_1[1])**2)
            +
            ((c1_point_3[0]-c1_point_2[0])**2)
        )
    ), 
    linestyle="dashed", 
)

c1_leg1 = ax_2ps[0].legend(
    [c1_fig_2ps_g1_p1, c1_fig_2ps_g1_p2, c1_fig_2ps_g1_p3, c1_fig_2ps_g1_p4, c1_fig_2ps_g1_p5, c1_fig_2ps_g1_p6], 
    [
        "point # {:}".format(temp_counter+1), 
        "point # {:}".format(temp_counter+2), 
        "self-extended point", 
        "base: {:.2f}m".format(abs(c1_point_3[0]-c1_point_2[0])), 
        "perpendicular: {:.2f}m".format(abs(c1_point_3[1]-c1_point_1[1])), 
        "hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c1_point_3[1]-c1_point_1[1])**2)
            +
            ((c1_point_3[0]-c1_point_2[0])**2)
        ))
    ], 
    loc="upper center"
)

ax_2ps[0].add_artist(c1_leg1)

In [ ]:
print (
    "E/W direction:\t{:}".format(c1_dir_x)
)
c1_x2_pred_temp = 0
if(c1_dir_x=="East"):
    c1_x2_pred_temp = c1_point_1[0] + dt*c1_point_1[2]
if(c1_dir_x=="West"):
    c1_x2_pred_temp = c1_point_1[0] - dt*c1_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c1_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c1_point_1[3], 
    get_sub("2"), c1_point_2[3])
) 
print ("x{:}: {:}, x{:}: {:}, x{:}:{:}".format(
    get_sub("1"), c1_point_1[0], 
    get_sub("2"), c1_point_2[0], 
    get_sub("2,pr"), c1_x2_pred_temp)
)

In [ ]:
print (
    "N/S direction:\t{:}".format(c1_dir_y)
)
c1_y2_pred_temp = 0
if(c1_dir_y=="North"):
    c1_y2_pred_temp = c1_point_1[1] + dt*c1_point_1[2]
if(c1_dir_y=="South"):
    c1_y2_pred_temp = c1_point_1[1] - dt*c1_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c1_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c1_point_1[3], 
    get_sub("2"), c1_point_2[3])
) 
print ("y{:}: {:}, y{:}: {:}, y{:}:{:}".format(
    get_sub("1"), c1_point_1[1], 
    get_sub("2"), c1_point_2[1], 
    get_sub("2,pr"), c1_y2_pred_temp)
)

### 1(b) Finding new location between two consecutive coordinates via angle

In [ ]:
print (veh1501_records_temp2)
print (veh1501_records_temp3)

In [ ]:
c2_dif_x = difference_in_unit(veh1501_records_temp2["x"].values[0], veh1501_records_temp3["x"].values[0])
c2_dir_x = "East" if c2_dif_x>0 else "West"
c2_dif_y = difference_in_unit(veh1501_records_temp2["y"].values[0], veh1501_records_temp3["y"].values[0])
c2_dir_y = "North" if c2_dif_y>0 else "South"
c2_dif_a = difference_in_unit(veh1501_records_temp2["angle"].values[0], veh1501_records_temp3["angle"].values[0])
c2_dif_s = difference_in_unit(veh1501_records_temp2["speed"].values[0], veh1501_records_temp3["speed"].values[0])

print (
    "Direction:\t{:}-{:}".format(c2_dir_y, c2_dir_x)
)
print (
    "(m)\t", "{:.20f}\t\t".format(c2_dif_x), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp3["x"].values[0], veh1501_records_temp2["x"].values[0]), 
)
print (
    "(m)\t", "{:.20f}\t".format(c2_dif_y), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp3["y"].values[0], veh1501_records_temp2["y"].values[0]), 
)
print (
    "(\u00B0)\t", "{:.20f}\t".format(c2_dif_a), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp3["angle"].values[0], veh1501_records_temp2["angle"].values[0]), 
)
print (
    "(m/s)\t", "{:.20f}\t\t".format(c2_dif_s), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp3["speed"].values[0], veh1501_records_temp2["speed"].values[0]), 
)

In [ ]:
c2_point_1 = [
    veh1501_records_temp2["x"].values[0], 
    veh1501_records_temp2["y"].values[0], 
    veh1501_records_temp2["speed"].values[0], 
    veh1501_records_temp2["angle"].values[0] 
]
c2_point_2 = [
    veh1501_records_temp3["x"].values[0], 
    veh1501_records_temp3["y"].values[0], 
    veh1501_records_temp3["speed"].values[0], 
    veh1501_records_temp3["angle"].values[0] 
]

if(c2_dir_y=="South"):
    c2_point_3_y = c2_point_2[1]
    print ("extended-point (y) should be taken from point_2")
if(c1_dir_y=="North"):
    c2_point_3_y = c2_point_2[1]
    print ("extended-point (y) should be taken from point_1")

if(c2_dir_x=="East"):
    c2_point_3_x = c2_point_1[0]
    print ("extended-point (x) should be taken from point_1")
if(c1_dir_x=="West"):
    c2_point_3_x = c2_point_1[0]
    print ("extended-point (x) should be taken from point_2")

c2_point_3 = [c2_point_1[0], c2_point_2[1]]
print (c2_point_3)
c2_point_3 = [c2_point_3_x, c2_point_3_y]
print (c2_point_3)

In [ ]:
c2_fig_2ps_g2_p1, = ax_2ps[0].plot(
    c2_point_1[0], 
    c2_point_1[1], 
    "ro", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+2)
)

c2_fig_2ps_g2_p2, = ax_2ps[0].plot(
    c2_point_2[0], 
    c2_point_2[1], 
    "go", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+3)
)

c2_fig_2ps_g2_p3, = ax_2ps[0].plot(
    c2_point_3[0], 
    c2_point_3[1], 
    "ys", 
    markersize=MARKER_SZ+4, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="self-extended point", 
)

c2_fig_2ps_g2_p4, = ax_2ps[0].plot(
    [c2_point_2[0], c2_point_3[0]], 
    [c2_point_2[1], c2_point_3[1]], 
    "c", 
    label="base: {:.2f}m".format(abs(c2_point_3[0]-c2_point_2[0])) 
)

c2_fig_2ps_g2_p5, = ax_2ps[0].plot(
    [c2_point_1[0], c2_point_3[0]], 
    [c2_point_1[1], c2_point_3[1]], 
    "m", 
    label="perpendicular: {:.2f}m".format(abs(c2_point_3[1]-c2_point_1[1])) 
)

c2_fig_2ps_g2_p6, = ax_2ps[0].plot(
    [c2_point_1[0], c2_point_2[0]], 
    [c2_point_1[1], c2_point_2[1]], 
    "r", 
    label="hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c2_point_3[1]-c2_point_1[1])**2)
            +
            ((c2_point_3[0]-c2_point_2[0])**2)
        )
    ), 
    linestyle="dashed", 
)

c2_leg2 = ax_2ps[0].legend(
    [c2_fig_2ps_g2_p1, c2_fig_2ps_g2_p2, c2_fig_2ps_g2_p3, c2_fig_2ps_g2_p4, c2_fig_2ps_g2_p5, c2_fig_2ps_g2_p6], 
    [
        "point # {:}".format(temp_counter+2), 
        "point # {:}".format(temp_counter+3), 
        "self-extended point", 
        "base: {:.2f}m".format(abs(c2_point_3[0]-c2_point_2[0])), 
        "perpendicular: {:.2f}m".format(abs(c2_point_3[1]-c2_point_1[1])), 
        "hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c2_point_3[1]-c2_point_1[1])**2)
            +
            ((c2_point_3[0]-c2_point_2[0])**2)
        ))
    ], 
    loc="center right"
)

ax_2ps[0].add_artist(c2_leg2)

In [ ]:
c_fig_2ps

In [ ]:
print (
    "E/W direction:\t{:}".format(c2_dir_x)
)
c2_x2_pred_temp = 0
if(c2_dir_x=="East"):
    c2_x2_pred_temp = c2_point_1[0] + dt*c2_point_1[2]
if(c1_dir_x=="West"):
    c2_x2_pred_temp = c2_point_1[0] - dt*c2_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c2_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c2_point_1[3], 
    get_sub("2"), c2_point_2[3])
) 
print ("x{:}: {:}, x{:}: {:}, x{:}:{:}".format(
    get_sub("1"), c2_point_1[0], 
    get_sub("2"), c2_point_2[0], 
    get_sub("2,pr"), c2_x2_pred_temp)
)

In [ ]:
print (
    "N/S direction:\t{:}".format(c1_dir_y)
)
c2_y2_pred_temp = 0
if(c2_dir_y=="North"):
    c2_y2_pred_temp = c2_point_1[1] + dt*c2_point_1[2]
if(c2_dir_y=="South"):
    c2_y2_pred_temp = c2_point_1[1] - dt*c2_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c2_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c2_point_1[3], 
    get_sub("2"), c2_point_2[3])
) 
print ("y{:}: {:}, y{:}: {:}, y{:}:{:}".format(
    get_sub("1"), c2_point_1[1], 
    get_sub("2"), c2_point_2[1], 
    get_sub("2,pr"), c2_y2_pred_temp)
)

### 2(a) Finding new location between two consecutive coordinates via angle

In [ ]:
print (veh1501_records_temp4)
print (veh1501_records_temp5)

In [ ]:
c3_dif_x = difference_in_unit(veh1501_records_temp4["x"].values[0], veh1501_records_temp5["x"].values[0])
c3_dir_x = "East" if c3_dif_x>0 else "West"
c3_dif_y = difference_in_unit(veh1501_records_temp4["y"].values[0], veh1501_records_temp5["y"].values[0])
c3_dir_y = "North" if c3_dif_y>0 else "South"
c3_dif_a = difference_in_unit(veh1501_records_temp4["angle"].values[0], veh1501_records_temp5["angle"].values[0])
c3_dif_s = difference_in_unit(veh1501_records_temp4["speed"].values[0], veh1501_records_temp5["speed"].values[0])

print (
    "Direction:\t{:}-{:}".format(c3_dir_y, c3_dir_x)
)
print (
    "(m)\t", "{:.20f}\t".format(c3_dif_x), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp4["x"].values[0], veh1501_records_temp5["x"].values[0]), 
)
print (
    "(m)\t", "{:.20f}\t".format(c3_dif_y), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp4["y"].values[0], veh1501_records_temp5["y"].values[0]), 
)
print (
    "(\u00B0)\t", "{:.20f}\t".format(c3_dif_a), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp4["angle"].values[0], veh1501_records_temp5["angle"].values[0]), 
)
print (
    "(m/s)\t", "{:.20f}\t".format(c3_dif_s), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp4["speed"].values[0], veh1501_records_temp5["speed"].values[0]), 
)

In [ ]:
c3_point_1 = [
    veh1501_records_temp4["x"].values[0], 
    veh1501_records_temp4["y"].values[0], 
    veh1501_records_temp4["speed"].values[0], 
    veh1501_records_temp4["angle"].values[0] 
]
c3_point_2 = [
    veh1501_records_temp5["x"].values[0], 
    veh1501_records_temp5["y"].values[0], 
    veh1501_records_temp5["speed"].values[0], 
    veh1501_records_temp5["angle"].values[0] 
]

if(c3_dir_y=="South"):
    c3_point_3_y = c3_point_2[1]
    print ("extended-point (y) should be taken from point_2")
if(c3_dir_y=="North"):
    c3_point_3_y = c3_point_2[1]
    print ("extended-point (y) should be taken from point_1")

if(c3_dir_x=="East"):
    c3_point_3_x = c3_point_1[0]
    print ("extended-point (x) should be taken from point_1")
if(c3_dir_x=="West"):
    c3_point_3_x = c3_point_1[0]
    print ("extended-point (x) should be taken from point_2")

c3_point_3 = [c3_point_1[0], c3_point_2[1]]
print (c3_point_3)
c3_point_3 = [c3_point_3_x, c3_point_3_y]
print (c3_point_3)

In [ ]:
c3_fig_2ps_g3_p1, = ax_2ps[1].plot(
    c3_point_1[0], 
    c3_point_1[1], 
    "ro", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+4)
)

c3_fig_2ps_g3_p2, = ax_2ps[1].plot(
    c3_point_2[0], 
    c3_point_2[1], 
    "go", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+5)
)

c3_fig_2ps_g3_p3, = ax_2ps[1].plot(
    c3_point_3[0], 
    c3_point_3[1], 
    "ys", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="self-extended point", 
)

c3_fig_2ps_g3_p4, = ax_2ps[1].plot(
    [c3_point_2[0], c3_point_3[0]], 
    [c3_point_2[1], c3_point_3[1]], 
    "c", 
    label="base: {:.2f}m".format(abs(c3_point_3[0]-c3_point_2[0])) 
)

c3_fig_2ps_g3_p5, = ax_2ps[1].plot(
    [c3_point_1[0], c3_point_3[0]], 
    [c3_point_1[1], c3_point_3[1]], 
    "m", 
    label="perpendicular: {:.2f}m".format(abs(c3_point_3[1]-c3_point_1[1])) 
)

c3_fig_2ps_g3_p6, = ax_2ps[1].plot(
    [c3_point_1[0], c3_point_2[0]], 
    [c3_point_1[1], c3_point_2[1]], 
    "r", 
    label="hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c3_point_3[1]-c3_point_1[1])**2)
            +
            ((c3_point_3[0]-c3_point_2[0])**2)
        )
    ), 
    linestyle="dashed", 
)

c3_leg3 = ax_2ps[1].legend(
    [c3_fig_2ps_g3_p1, c3_fig_2ps_g3_p2, c3_fig_2ps_g3_p3, c3_fig_2ps_g3_p4, c3_fig_2ps_g3_p5, c3_fig_2ps_g3_p6], 
    [
        "point # {:}".format(temp_counter+4), 
        "point # {:}".format(temp_counter+5), 
        "self-extended point", 
        "base: {:.2f}m".format(abs(c3_point_3[0]-c3_point_2[0])), 
        "perpendicular: {:.2f}m".format(abs(c3_point_3[1]-c3_point_1[1])), 
        "hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c3_point_3[1]-c3_point_1[1])**2)
            +
            ((c3_point_3[0]-c3_point_2[0])**2)
        ))
    ], 
    loc="center right"
)

ax_2ps[1].add_artist(c3_leg3)

In [ ]:
c_fig_2ps

In [ ]:
print (
    "E/W direction:\t{:}".format(c2_dir_x)
)
c3_x2_pred_temp = 0
if(c3_dir_x=="East"):
    c3_x2_pred_temp = c3_point_1[0] + dt*c3_point_1[2]
if(c1_dir_x=="West"):
    c3_x2_pred_temp = c3_point_1[0] - dt*c3_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c3_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c3_point_1[3], 
    get_sub("2"), c3_point_2[3])
) 
print ("x{:}: {:}, x{:}: {:}, x{:}:{:}".format(
    get_sub("1"), c3_point_1[0], 
    get_sub("2"), c3_point_2[0], 
    get_sub("2,pr"), c3_x2_pred_temp)
)

In [ ]:
print (
    "N/S direction:\t{:}".format(c1_dir_y)
)
c3_y2_pred_temp = 0
if(c3_dir_y=="North"):
    c3_y2_pred_temp = c3_point_1[1] + dt*c3_point_1[2]
if(c3_dir_y=="South"):
    c3_y2_pred_temp = c3_point_1[1] - dt*c3_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c3_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c3_point_1[3], 
    get_sub("2"), c3_point_2[3])
) 
print ("y{:}: {:}, y{:}: {:}, y{:}:{:}".format(
    get_sub("1"), c3_point_1[1], 
    get_sub("2"), c3_point_2[1], 
    get_sub("2,pr"), c3_y2_pred_temp)
)

### 2(b) Finding new location between two consecutive coordinates via angle

In [ ]:
print (veh1501_records_temp5)
print (veh1501_records_temp6)

In [ ]:
c4_dif_x = difference_in_unit(veh1501_records_temp5["x"].values[0], veh1501_records_temp6["x"].values[0])
c4_dir_x = "East" if c4_dif_x>0 else "West"
c4_dif_y = difference_in_unit(veh1501_records_temp5["y"].values[0], veh1501_records_temp6["y"].values[0])
c4_dir_y = "North" if c4_dif_y>0 else "South"
c4_dif_a = difference_in_unit(veh1501_records_temp5["angle"].values[0], veh1501_records_temp6["angle"].values[0])
c4_dif_s = difference_in_unit(veh1501_records_temp5["speed"].values[0], veh1501_records_temp6["speed"].values[0])

print (
    "Direction:\t{:}-{:}".format(c4_dir_y, c4_dir_x)
)
print (
    "(m)\t", "{:.20f}\t".format(c4_dif_x), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp5["x"].values[0], veh1501_records_temp6["x"].values[0]), 
)
print (
    "(m)\t", "{:.20f}\t\t".format(c4_dif_y), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp5["y"].values[0], veh1501_records_temp6["y"].values[0]), 
)
print (
    "(\u00B0)\t", "{:.20f}\t".format(c4_dif_a), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp5["angle"].values[0], veh1501_records_temp6["angle"].values[0]), 
)
print (
    "(m/s)\t", "{:.20f}\t\t".format(c4_dif_s), 
    "({:.2f}-{:.2f})".format(veh1501_records_temp5["speed"].values[0], veh1501_records_temp6["speed"].values[0]), 
)

In [ ]:
c4_point_1 = [
    veh1501_records_temp5["x"].values[0], 
    veh1501_records_temp5["y"].values[0], 
    veh1501_records_temp5["speed"].values[0], 
    veh1501_records_temp5["angle"].values[0] 
]
c4_point_2 = [
    veh1501_records_temp6["x"].values[0], 
    veh1501_records_temp6["y"].values[0], 
    veh1501_records_temp6["speed"].values[0], 
    veh1501_records_temp6["angle"].values[0] 
]

if(c4_dir_y=="South"):
    c4_point_3_y = c4_point_2[1]
    print ("extended-point (y) should be taken from point_2")
if(c4_dir_y=="North"):
    c4_point_3_y = c4_point_2[1]
    print ("extended-point (y) should be taken from point_1")

if(c4_dir_x=="East"):
    c4_point_3_x = c4_point_1[0]
    print ("extended-point (x) should be taken from point_1")
if(c4_dir_x=="West"):
    c4_point_3_x = c4_point_1[0]
    print ("extended-point (x) should be taken from point_2")

c4_point_3 = [c4_point_1[0], c4_point_2[1]]
print (c4_point_3)
c4_point_3 = [c4_point_3_x, c4_point_3_y]
print (c4_point_3)

In [ ]:
c4_fig_2ps_g4_p1, = ax_2ps[1].plot(
    c4_point_1[0], 
    c4_point_1[1], 
    "ro", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+5)
)

c4_fig_2ps_g4_p2, = ax_2ps[1].plot(
    c4_point_2[0], 
    c4_point_2[1], 
    "go", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="point # {:}".format(temp_counter+6)
)

c4_fig_2ps_g4_p3, = ax_2ps[1].plot(
    c4_point_3[0], 
    c4_point_3[1], 
    "ys", 
    markersize=MARKER_SZ+5, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.5, 
    label="self-extended point", 
)

c4_fig_2ps_g4_p4, = ax_2ps[1].plot(
    [c4_point_2[0], c4_point_3[0]], 
    [c4_point_2[1], c4_point_3[1]], 
    "c", 
    label="base: {:.2f}m".format(abs(c4_point_3[0]-c4_point_2[0])) 
)

c4_fig_2ps_g4_p5, = ax_2ps[1].plot(
    [c4_point_1[0], c4_point_3[0]], 
    [c4_point_1[1], c4_point_3[1]], 
    "m", 
    label="perpendicular: {:.2f}m".format(abs(c4_point_3[1]-c4_point_1[1])) 
)

c4_fig_2ps_g4_p6, = ax_2ps[1].plot(
    [c4_point_1[0], c4_point_2[0]], 
    [c4_point_1[1], c4_point_2[1]], 
    "r", 
    label="hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c4_point_3[1]-c4_point_1[1])**2)
            +
            ((c4_point_3[0]-c4_point_2[0])**2)
        )
    ), 
    linestyle="dashed", 
)

c4_leg4 = ax_2ps[1].legend(
    [c4_fig_2ps_g4_p1, c4_fig_2ps_g4_p2, c4_fig_2ps_g4_p3, c4_fig_2ps_g4_p4, c4_fig_2ps_g4_p5, c4_fig_2ps_g4_p6], 
    [
        "point # {:}".format(temp_counter+5), 
        "point # {:}".format(temp_counter+6), 
        "self-extended point", 
        "base: {:.2f}m".format(abs(c4_point_3[0]-c4_point_2[0])), 
        "perpendicular: {:.2f}m".format(abs(c4_point_3[1]-c4_point_1[1])), 
        "hypotenuse: {:.2f}m".format(
        math.sqrt(
            ((c4_point_3[1]-c4_point_1[1])**2)
            +
            ((c4_point_3[0]-c4_point_2[0])**2)
        ))
    ], 
    loc="center left"
)

ax_2ps[1].add_artist(c4_leg4)

In [ ]:
c_fig_2ps

In [ ]:
c4_x2_pred_temp = c4_point_1[0] - dt*c4_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c4_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c4_point_1[3], 
    get_sub("2"), c4_point_2[3])
) 
print ("x{:}: {:}, x{:}: {:}, x{:}:{:}".format(
    get_sub("1"), c4_point_1[0], 
    get_sub("2"), c4_point_2[0], 
    get_sub("2,pr"), c4_x2_pred_temp)
)

In [ ]:
c4_y2_pred_temp = c4_point_1[1] - dt*c4_point_1[2]

print ("s{:}: {:}".format(get_sub("1"), c4_point_1[2]))
print ("\u0394t: {:}".format(dt))
print ("\u03F4{:}: {:}\u00B0, \u03F4{:}: {:}\u00B0".format(
    get_sub("1"), c4_point_1[3], 
    get_sub("2"), c4_point_2[3])
) 
print ("y{:}: {:}, y{:}: {:}, y{:}:{:}".format(
    get_sub("1"), c4_point_1[1], 
    get_sub("2"), c4_point_2[1], 
    get_sub("2,pr"), c4_y2_pred_temp)
)

## Dividing all [east / turn / south / turn / west] movements

In [ ]:
def prexict(x, y, a, s, dir="East"): 
    if (dir == "East"): 
        return x+s*dt
    elif(dir == "West"): 
        return x-s*dt

In [ ]:
def preyict(x, y, a, s, dir="North"): 
    if (dir == "North"): 
        return y+s*dt
    elif(dir == "South"): 
        return y-s*dt

In [ ]:
# East movement
veh1501_records_temp_east = veh1501_records[veh1501_records["time"] >= 8029]
veh1501_records_temp_east = veh1501_records_temp_east[veh1501_records_temp_east["time"] <=8047]

# East to South (Angle) movement
veh1501_records_temp_east_south1 = veh1501_records_temp[veh1501_records_temp["time"] == 8047]
veh1501_records_temp_east_south2 = veh1501_records_temp[veh1501_records_temp["time"] == 8048]
veh1501_records_temp_east_south3 = veh1501_records_temp[veh1501_records_temp["time"] == 8049]

# South movement
veh1501_records_temp_south = veh1501_records[veh1501_records["time"] >= 8049]
veh1501_records_temp_south = veh1501_records_temp_south[veh1501_records_temp_south["time"] <= 8063]

# South to West (Angle)
veh1501_records_temp_south_west4 = veh1501_records_temp[veh1501_records_temp["time"] == 8063]
veh1501_records_temp_south_west5 = veh1501_records_temp[veh1501_records_temp["time"] == 8064]
veh1501_records_temp_south_west6 = veh1501_records_temp[veh1501_records_temp["time"] == 8065]

# West movement
veh1501_records_temp_west = veh1501_records[veh1501_records["time"] >= 8065]
veh1501_records_temp_west = veh1501_records_temp_west[veh1501_records_temp_west["time"] <= 8087]

### East movement

In [ ]:
print (veh1501_records_temp_east)

In [ ]:
fig_veh1501_east, ax_veh1501_east = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1501_east_actual, = ax_veh1501_east.plot(
    veh1501_records_temp_east["x"], 
    veh1501_records_temp_east["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1501_east_actual_legend = ax_veh1501_east.legend(
    [fig_veh1501_east_actual], 
    [
        "Vehicle 1501 (East-movement)" 
    ], 
    loc="upper center"
)

ax_veh1501_east.add_artist(fig_veh1501_east_actual_legend)

### Predicting using Kalman filter (East)

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["x", "y"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1501_records_temp_east.iterrows(): 
    next_index_x = prexict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"], "East") 
    next_index_y = veh1501_records[veh1501_records["time"] == vehicle["time"]+1]["y"].values[0]
    print (
        "{:}, {:}, {:.2f}, {:.2f}, {:.2f}, {:.2f}".format(
            vehicle["time"], 
            vehicle["id"], 
            vehicle["x"], 
            vehicle["y"], 
            vehicle["angle"], 
            vehicle["speed"] 
        )
    )
    dataset_rows.append({
        "x": next_index_x, 
        "y": next_index_y, 
    })

veh1501_records_temp_east_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1501_east_predicted, = ax_veh1501_east.plot(
    veh1501_records_temp_east_pred["x"], 
    veh1501_records_temp_east_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
)

fig_veh1501_east_legend = ax_veh1501_east.legend(
    [fig_veh1501_east_actual, fig_veh1501_east_predicted], 
    [
        "Vehicle 1501 Actual (East-movement)", 
        "Vehicle 1501 Predicted (East-movement)" 
    ], 
    loc="upper center"
)

ax_veh1501_east.add_artist(fig_veh1501_east_legend)

fig_veh1501_east

### East to South (Angle) movement

In [ ]:
veh1501_records_temp_east_south = pd.concat(
    [
        veh1501_records_temp_east_south1, 
        veh1501_records_temp_east_south2, 
        veh1501_records_temp_east_south3
    ], 
    ignore_index=True
)
print (veh1501_records_temp_east_south)

In [ ]:
fig_veh1501_east_south, ax_veh1501_east_south = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(13.5,9)
)

fig_veh1501_east_south_actual, = ax_veh1501_east_south.plot(
    veh1501_records_temp_east_south["x"], 
    veh1501_records_temp_east_south["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1501_east_south_actual_legend = ax_veh1501_east_south.legend(
    [fig_veh1501_east_south_actual], 
    [
        "Vehicle 1501 (East-South-movement)" 
    ], 
    loc="best"
)

ax_veh1501_east_south.add_artist(fig_veh1501_east_south_actual_legend)

### Predicting using Kalman filter (East-South)

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["x", "y"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1501_records_temp_east_south.iterrows():
    next_index_x, next_index_y = preaict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"])
    dataset_rows.append({
        "x": next_index_x, 
        "y": next_index_y, 
    })
veh1501_records_temp_east_south_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1501_east_south_pred, = ax_veh1501_east_south.plot(
    veh1501_records_temp_east_south_pred["x"], 
    veh1501_records_temp_east_south_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
) 

fig_veh1501_east_south_legend = ax_veh1501_east_south.legend(
    [fig_veh1501_east_south_actual, fig_veh1501_east_south_pred], 
    [
        "Vehicle 1501 Actual (East-South-movement)", 
        "Vehicle 1501 Predicted (East-South-movement)" 
    ], 
    loc="best"
) 

ax_veh1501_east_south.add_artist(fig_veh1501_east_south_legend) 

fig_veh1501_east_south

### South movement

In [ ]:
print (veh1501_records_temp_south)

In [ ]:
fig_veh1501_south, ax_veh1501_south = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1501_south_actual, = ax_veh1501_south.plot(
    veh1501_records_temp_south["x"], 
    veh1501_records_temp_south["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1501_south_actual_legend = ax_veh1501_south.legend(
    [fig_veh1501_south_actual], 
    [
        "Vehicle 1501 (South-movement)" 
    ], 
    loc="upper center"
)

ax_veh1501_south.add_artist(fig_veh1501_south_actual_legend)

### Predicting using Kalman filter (South)

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["x", "y"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1501_records_temp_south.iterrows():
    next_index_x = veh1501_records[veh1501_records["time"] == vehicle["time"]+1]["x"].values[0]
    next_index_y = preyict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"], "South") 
    print (
        "{:}, {:}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}".format(
            vehicle["time"], 
            vehicle["id"], 
            vehicle["x"], next_index_x, 
            vehicle["y"], 
            vehicle["angle"], 
            vehicle["speed"] 
        )
    )
    dataset_rows.append({
        "x": next_index_x, 
        "y": next_index_y, 
    })

veh1501_records_temp_south_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1501_south_predicted, = ax_veh1501_south.plot(
    veh1501_records_temp_south_pred["x"], 
    veh1501_records_temp_south_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
) 

fig_veh1501_south_legend = ax_veh1501_south.legend(
    [fig_veh1501_south_actual, fig_veh1501_south_predicted], 
    [
        "Vehicle 1501 Actual (South-movement)", 
        "Vehicle 1501 Predicted (South-movement)" 
    ], 
    loc="upper center"
) 

ax_veh1501_south.add_artist(fig_veh1501_south_legend) 

fig_veh1501_south 

### Angle ()

In [ ]:
print (veh1501_records_temp_south_west4)
print (veh1501_records_temp_south_west5)
print (veh1501_records_temp_south_west6)

### West movement

In [ ]:
print (veh1501_records_temp_west)

In [ ]:
fig_veh1501_west, ax_veh1501_west = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1501_west_actual, = ax_veh1501_west.plot(
    veh1501_records_temp_west["x"], 
    veh1501_records_temp_west["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1501_west_actual_legend = ax_veh1501_west.legend(
    [fig_veh1501_west_actual], 
    [
        "Vehicle 1501 (West-movement)" 
    ], 
    loc="upper center"
)

ax_veh1501_west.add_artist(fig_veh1501_west_actual_legend)

### Predicting using Kalman filter (West)

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["x", "y"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1501_records_temp_west.iterrows():
    next_index_x = prexict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"], "West") 
    next_index_y = veh1501_records[veh1501_records["time"] == vehicle["time"]+1]["y"].values[0]
    print (
        "{:}, {:}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}".format(
            vehicle["time"], 
            vehicle["id"], 
            vehicle["x"], next_index_x, 
            vehicle["y"], 
            vehicle["angle"], 
            vehicle["speed"] 
        )
    )
    dataset_rows.append({
        "x": next_index_x, 
        "y": next_index_y, 
    })

veh1501_records_temp_west_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1501_west_predicted, = ax_veh1501_west.plot(
    veh1501_records_temp_west_pred["x"], 
    veh1501_records_temp_west_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
) 

fig_veh1501_west_legend = ax_veh1501_west.legend(
    [fig_veh1501_west_actual, fig_veh1501_west_predicted], 
    [
        "Vehicle 1501 Actual (West-movement)", 
        "Vehicle 1501 Predicted (West-movement)" 
    ], 
    loc="upper center"
) 

ax_veh1501_west.add_artist(fig_veh1501_west_legend) 

fig_veh1501_west

## Predicting all together for Vehicle # veh1501

In [ ]:
veh1501_records = df_org[df_org["id"] == "veh1501"]

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["x", "y"]
dataset_rows = []

In [ ]:
# for index, vehicle in veh1501_records.iterrows():
#     next_index_x, next_index_y = preaict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"])
#     dataset_rows.append({
#         "x": next_index_x, 
#         "y": next_index_y, 
#     }) 
#     print (vehicle["x"], next_index_x) 
#     print (vehicle["y"], next_index_y)
# veh1501_records_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
for index, vehicle in veh1501_records.iterrows():
    next_index_x, next_index_y = preaict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"])
    dataset_rows.append({
        "time": vehicle["time"]+1, 
        "x": next_index_x, 
        "y": next_index_y, 
    }) 
    print (vehicle["x"], next_index_x) 
    print (vehicle["y"], next_index_y)
veh1501_records_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1501_all, ax_veh1501_all = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1501_all_actual, = ax_veh1501_all.plot(
    veh1501_records["x"], 
    veh1501_records["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1501_all_pred, = ax_veh1501_all.plot(
    veh1501_records_pred["x"], 
    veh1501_records_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
)

fig_veh1501_all_legend = ax_veh1501_all.legend(
    [fig_veh1501_all_actual, fig_veh1501_all_pred], 
    [
        "Vehicle 1501 Actual (All-movement)", 
        "Vehicle 1501 Predicted (All-movement)" 
    ], 
    loc="best"
)

ax_veh1501_all.add_artist(fig_veh1501_all_legend)

## Predicting all together for Vehicle # veh1502

In [ ]:
veh1502_records = df_org[df_org["id"] == "veh1502"]

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["time", "x", "y"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1502_records.iterrows():
    next_index_x, next_index_y = preaict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"])
    dataset_rows.append({
        "time": vehicle["time"]+1, 
        "x": next_index_x, 
        "y": next_index_y, 
    }) 
    print (vehicle["x"], next_index_x) 
    print (vehicle["y"], next_index_y)
veh1502_records_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1502_all, ax_veh1502_all = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1502_all_actual, = ax_veh1502_all.plot(
    veh1502_records["x"], 
    veh1502_records["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1502_all_pred, = ax_veh1502_all.plot(
    veh1502_records_pred["x"], 
    veh1502_records_pred["y"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
)

fig_veh1502_all_legend = ax_veh1502_all.legend(
    [
        fig_veh1502_all_actual, fig_veh1502_all_pred
    ], 
    [
        "Vehicle 1502 Actual (All-movement)", 
        "Vehicle 1502 Predicted (All-movement)" 
    ], 
    loc="best"
)

ax_veh1502_all.add_artist(fig_veh1502_all_legend)

## Predicting all together for Vehicle # veh1503

In [ ]:
def detect_direction(angle=0.00): 
    if (angle == 0.00): 
        return "None", "North" 
    elif (angle>0.00 and angle<90.00): #
        return "East", "North" 
    elif (angle == 90.00): 
        return "East", "None" 
    elif (angle>90.00 and angle<180.00): #
        return "East", "South" 
    elif (angle == 180.00): 
        return "None", "South" 
    elif (angle>180.00 and angle<270.00): #
        return "West", "South" 
    elif (angle == 270.00): 
        return "West", "None" 
    elif (angle>270.00 and angle<360.00): #
        return "West", "North" 

In [ ]:
def find_min_max_weight(x_dir, y_dir): 
    if (x_dir == "East" and y_dir == "North"): 
        return 0, 90
    elif (x_dir == "East" and y_dir == "South"): 
        return 90, 180
    elif (x_dir == "West" and y_dir == "South"): 
        return 180, 270
    elif (x_dir == "West" and y_dir == "North"): 
        return 270, 360
    else: 
        return 0.0, 0.0

In [ ]:
def preaict(x, y, a, s): 
    x_dir , y_dir = detect_direction(a)
    minweight_angle, maxweight_angle = find_min_max_weight(x_dir, y_dir)
    weight_range = maxweight_angle-minweight_angle
    xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
    print ("{:}, {:}, {:.2f}, {:.2f}, {:.2f}".format(
        x_dir, y_dir, a, xweight_angle, yweight_angle
    ))
    if (x_dir == "None" or y_dir == "None"): 
        if (x_dir == "None"): 
            yweight_angle = 1
            if (y_dir == "North"): 
                #print ("None-North: {:.2f}".format(a))
                x_pred, y_pred = x, y+yweight_angle*dt*s
            elif (y_dir == "South"): 
                #print ("None-South: {:.2f}".format(a))
                x_pred, y_pred = x, y-yweight_angle*dt*s
        elif (y_dir == "None"): 
            xweight_angle = 1
            if (x_dir == "East"): 
                #print ("East-None: {:.2f}".format(a))
                x_pred, y_pred = x+yweight_angle*dt*s, y
            elif (x_dir == "West"): 
                #print ("West-None: {:.2f}".format(a))
                x_pred, y_pred = x-yweight_angle*dt*s, y
    elif (x_dir == "East" and y_dir == "North"): 
        #print ("East-North: {:.2f}".format(a))
        xweight_angle, yweight_angle = (a-minweight_angle)/weight_range, (maxweight_angle-a)/weight_range
        x_pred, y_pred = x+xweight_angle*dt*s, y+yweight_angle*dt*s
    elif (x_dir == "East" and y_dir == "South"): 
        #print ("East-South: {:.2f}".format(a))
        xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
        x_pred, y_pred = x+xweight_angle*dt*s, y-yweight_angle*dt*s
    elif (x_dir == "West" and y_dir == "South"): 
        #print ("West-South: {:.2f}".format(a))
        xweight_angle, yweight_angle = (a-minweight_angle)/weight_range, (maxweight_angle-a)/weight_range
        x_pred, y_pred = x-xweight_angle*dt*s, y-yweight_angle*dt*s
    elif (x_dir == "West" and y_dir == "North"): 
        #print ("West-North: {:.2f}".format(a))
        xweight_angle, yweight_angle = (maxweight_angle-a)/weight_range, (a-minweight_angle)/weight_range
        x_pred, y_pred = x-xweight_angle*dt*s, y+yweight_angle*dt*s
    return round(x_pred, 2), round(y_pred, 2)

In [ ]:
veh1503_records = df_org[df_org["id"] == "veh1503"]

In [ ]:
#dataset_cols = ["time", "id", "x", "y", "angle", "speed"]
dataset_cols = ["time", "xp", "yp"]
dataset_rows = []

In [ ]:
for index, vehicle in veh1503_records.iterrows():
    next_index_x, next_index_y = preaict(vehicle["x"], vehicle["y"], vehicle["angle"], vehicle["speed"])
    dataset_rows.append({
        "time": vehicle["time"]+1, 
        "xp": next_index_x, 
        "yp": next_index_y, 
    }) 
    print (vehicle["x"], next_index_x) 
    print (vehicle["y"], next_index_y)
veh1503_records_pred = pd.DataFrame(dataset_rows, columns = dataset_cols)

In [ ]:
fig_veh1503_all, ax_veh1503_all = plt.subplots(
    1, 
    # sharex=True, sharey=True, 
    figsize=(18,12)
)

fig_veh1503_all_actual, = ax_veh1503_all.plot(
    veh1503_records["x"], 
    veh1503_records["y"], 
    marker=".", 
    markersize=MARKER_SZ, 
)

fig_veh1503_all_pred, = ax_veh1503_all.plot(
    veh1503_records_pred["xp"], 
    veh1503_records_pred["yp"], 
    "ro", 
    markersize=MARKER_SZ, 
    mfc="None", 
    mew=2, 
    alpha=0.7, 
    linewidth=1.0, 
)

fig_veh1503_all_legend = ax_veh1503_all.legend(
    [
        fig_veh1503_all_actual, fig_veh1503_all_pred
    ], 
    [
        "Vehicle 1503 Actual (All-movement)", 
        "Vehicle 1503 Predicted (All-movement)" 
    ], 
    loc="best"
)

ax_veh1503_all.add_artist(fig_veh1503_all_legend)

## Error for vehicle 1501

In [ ]:
veh1501_records_actual1 = veh1501_records.drop(veh1501_records.index[0], inplace=False)

In [ ]:
veh1501_records_actual1

In [ ]:
veh1501_records_pred1 = veh1501_records_pred.drop(veh1501_records_pred.index[-1], inplace=False)

In [ ]:
veh1501_records_pred1

In [ ]:
veh1501_records_actual_pred_diff = pd.merge(
    veh1501_records_actual1.reset_index(), 
    veh1501_records_pred1.reset_index(), 
    on="time" 
)

veh1501_records_actual_pred_diff = veh1501_records_actual_pred_diff.drop(columns=["index_x", "index_y"])

veh1501_records_actual_pred_diff["xd"] = abs(
    round(veh1501_records_actual_pred_diff["x"]-veh1501_records_actual_pred_diff["xp"], 2)
)
veh1501_records_actual_pred_diff["yd"] = abs(
    veh1501_records_actual_pred_diff["y"]-veh1501_records_actual_pred_diff["yp"]
)

In [ ]:
veh1501_records_actual_pred_diff = veh1501_records_actual_pred_diff[[
#     "time", 
#     "id", 
    "angle", 
    "speed", 
    "x", 
    "xp", 
    "xd", 
    "y", 
    "yp", 
    "yd" 
]]

In [ ]:
veh1503_records_actual_pred_diff_sorted = veh1503_records_actual_pred_diff

pd.set_option("display.max_rows", veh1503_records_actual_pred_diff_sorted.shape[0])
print (veh1503_records_actual_pred_diff_sorted)
pd.set_option("display.max_rows", display_max_rows)

In [ ]:
error_data_veh1501 = error_xy_array(veh1501_records_actual1, veh1501_records_pred1)
error_data_veh1501 = error_data_veh1501.sort_values(["distance"], ascending=True)

In [ ]:
error_data_veh1501

## Error for vehicle 1502

In [ ]:
veh1502_records_actual1 = veh1502_records.drop(veh1502_records.index[0], inplace=False)

In [ ]:
veh1502_records_pred1 = veh1502_records_pred.drop(veh1502_records_pred.index[-1], inplace=False)

In [ ]:
veh1502_records_actual_pred_diff = pd.merge(
    veh1502_records_actual1.reset_index(), 
    veh1502_records_pred1.reset_index(), 
    on="time" 
)

veh1502_records_actual_pred_diff = veh1502_records_actual_pred_diff.drop(columns=["index_x", "index_y"])

veh1502_records_actual_pred_diff["xd"] = abs(
    round(veh1503_records_actual_pred_diff["x"]-veh1502_records_actual_pred_diff["xp"], 2)
)
veh1502_records_actual_pred_diff["yd"] = abs(
    veh1502_records_actual_pred_diff["y"]-veh1502_records_actual_pred_diff["yp"]
)

In [ ]:
veh1502_records_actual_pred_diff = veh1502_records_actual_pred_diff[[
#     "time", 
#     "id", 
    "angle", 
    "speed", 
    "x", 
    "xp", 
    "xd", 
    "y", 
    "yp", 
    "yd" 
]]

In [ ]:
veh1502_records_actual_pred_diff_sorted = veh1502_records_actual_pred_diff 

pd.set_option("display.max_rows", veh1502_records_actual_pred_diff_sorted.shape[0]) 
print (veh1502_records_actual_pred_diff_sorted) 
pd.set_option("display.max_rows", display_max_rows) 

In [ ]:
error_data_veh1502 = error_xy_array(veh1502_records_actual1, veh1502_records_pred1)
error_data_veh1502 = error_data_veh1502.sort_values(["distance"], ascending=True)

In [ ]:
error_data_veh1502

In [ ]:
# created my own RMSE function 
root_mean_square_error(error_data_veh1502) 

## Error for vehicle 1503

In [ ]:
veh1503_records_actual1 = veh1503_records.drop(veh1503_records.index[0], inplace=False)

In [ ]:
veh1503_records_pred1 = veh1503_records_pred.drop(veh1503_records_pred.index[-1], inplace=False)

In [ ]:
veh1503_records_actual_pred_diff = pd.merge(
    veh1503_records_actual1.reset_index(), 
    veh1503_records_pred1.reset_index(), 
    on="time" 
)

veh1503_records_actual_pred_diff = veh1503_records_actual_pred_diff.drop(columns=["index_x", "index_y"])

veh1503_records_actual_pred_diff["xd"] = abs(
    round(veh1503_records_actual_pred_diff["x"]-veh1503_records_actual_pred_diff["xp"], 2)
)
veh1503_records_actual_pred_diff["yd"] = abs(
    veh1503_records_actual_pred_diff["y"]-veh1503_records_actual_pred_diff["yp"]
)

In [ ]:
veh1503_records_actual_pred_diff = veh1503_records_actual_pred_diff[[
#     "time", 
#     "id", 
    "angle", 
    "speed", 
    "x", 
    "xp", 
    "xd", 
    "y", 
    "yp", 
    "yd" 
]]

In [ ]:
veh1503_records_actual_pred_diff_sorted = veh1503_records_actual_pred_diff

pd.set_option("display.max_rows", veh1503_records_actual_pred_diff_sorted.shape[0])
print (veh1503_records_actual_pred_diff_sorted)
pd.set_option("display.max_rows", display_max_rows)

In [ ]:
error_data_veh1503 = error_xy_array(veh1503_records_actual1, veh1503_records_pred1)
error_data_veh1503 = error_data_veh1503.sort_values(["distance"], ascending=True)

In [ ]:
error_data_veh1503

In [ ]:
# created my own RMSE function 
root_mean_square_error(error_data_veh1503) 